In [1]:
df_train= pd.read_csv(r"C:\Users\SARVESH\Documents\GitHub\House price prediction\train_clean_mod.csv")
df_test = pd.read_csv(r"C:\Users\SARVESH\Documents\GitHub\House price prediction\test_clean_mod.csv")

In [2]:
df_train.drop(['Id'] , axis = 1,inplace = True)
df_test.drop(['Id'] , axis = 1, inplace =True)

In [3]:
#Deleting outliers
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)

In [4]:
cols = df_train.corr().nlargest(150, 'SalePrice')['SalePrice'].index

In [5]:
df_train = df_train[cols]
df_test = df_test[cols]

In [6]:
ytrain = df_train['SalePrice']
df_train.drop(['SalePrice'], axis = 1, inplace = True)
df_test.drop(['SalePrice'],axis = 1,inplace = True)


In [7]:
from scipy import stats
from scipy.stats import norm , skew
numeric_feats = df_train.dtypes[df_train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = df_train[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
ImStucc,38.131361
RRAn,38.131361
PosN,38.131361
AsphShn,38.131361
Membran,38.131361
RRAe,38.131361
Roll,38.131361
NoSeWa,38.131361
Metal,38.131361
Other,38.131361


In [8]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    df_train[feat] = boxcox1p(df_train[feat], lam)
    df_test[feat] = boxcox1p(df_test[feat],lam)

There are 149 skewed numerical features to Box Cox transform


In [9]:
df_train.head()

,OverallQual,GrLivArea,TotalBsmtSF,GarageCars,1stFlrSF,GarageArea,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,LwQ,1.5Unf,P,Grav,SFoyer,HdBoard,2fmCon,Twnhs,BrDale,CompShg
0,2.440268,13.698888,11.692623,1.194318,11.692623,10.506271,1.194318,2.602594,14.187527,14.187527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463
1,2.259674,12.792276,12.792276,1.194318,12.792276,10.062098,1.194318,2.259674,14.145138,14.145138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463
2,2.440268,13.832085,11.892039,1.194318,11.892039,10.775536,1.194318,2.259674,14.184404,14.185966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463
3,2.440268,13.711364,11.354094,1.540963,12.013683,10.918253,0.730463,2.440268,14.047529,14.135652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463
4,2.602594,14.480029,12.510588,1.540963,12.510588,11.627708,1.194318,2.750250,14.182841,14.182841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463


In [10]:
df_test.head()

,OverallQual,GrLivArea,TotalBsmtSF,GarageCars,1stFlrSF,GarageArea,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,LwQ,1.5Unf,P,Grav,SFoyer,HdBoard,2fmCon,Twnhs,BrDale,CompShg
0,2.055642,11.818680,11.775114,0.730463,11.818680,11.259868,0.730463,2.055642,14.121376,14.121376,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.730463
1,2.259674,12.943735,12.943735,0.730463,12.943735,9.118181,0.730463,2.259674,14.116605,14.116605,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.730463
2,2.055642,13.551272,11.916131,1.194318,11.916131,10.179489,1.194318,2.259674,14.178149,14.179714,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.730463
3,2.259674,13.504453,11.910125,1.194318,11.910125,10.116035,1.194318,2.440268,14.179714,14.179714,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.730463
4,2.602594,12.833625,12.833625,1.194318,12.833625,10.302477,1.194318,2.055642,14.170316,14.170316,...,0.0,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,0.730463


In [11]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [12]:
import xgboost
regressor=xgboost.XGBRegressor()
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
## Hyper Parameter Optimization


n_estimators = [100, 500, 900, 1100, 1500 , 3000]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }
# Set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [ ]:
random_cv.best_estimator_

In [14]:
regressor=xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=2, min_child_weight=3, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1,verbosity=1)

In [15]:
regressor.fit(df_train,ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[16:53:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=2, min_child_weight=3, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [16]:
y_pred = regressor.predict(df_test)

In [17]:
y_pred

array([117797.836, 164662.02 , 183229.42 , ..., 188948.92 , 124553.125,
       231198.05 ], dtype=float32)

In [18]:
sub = pd.DataFrame(y_pred)
sub.to_csv(r'C:\Users\SARVESH\Documents\GitHub\House price prediction\minmax.csv',index = False)